In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import brewer2mpl
import colorsys
import math
import dendropy as dp
import json
import os

from datetime import datetime
from Bio import AlignIO, SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Levenshtein import distance
from itertools import combinations, product, permutations, combinations_with_replacement
from time import time
from __future__ import division
from collections import Counter, defaultdict
from copy import deepcopy
from random import shuffle, choice, sample
from scipy.stats.mstats import mquantiles
from scipy.stats import norm, expon, poisson, binom
from scipy.misc import comb
from IPython.display import Math
from networkx.readwrite import json_graph 

%matplotlib inline

In [8]:
G = nx.read_gpickle('20141103 All IRD Parallel Transmission Graph Non-Same Subtypes Removed.pkl')
for node, data in G.nodes(data=True):
    if 'host' not in data.keys():
        print(node)

In [7]:
subdir = 'subgraph_pkl'
subgraphs = dict()
for f in os.listdir(subdir):
    i = f.split(' ')[0]
    g = nx.read_gpickle('{0}/{1}'.format(subdir, f))
    subgraphs[int(i)] = g
# subgraphs = [g for g in nx.connected_component_subgraphs(G.to_undirected())]

In [9]:
# G.edges(data=True)

In [18]:
# Which H7N9 nodes have no in_edges?
for node, ndata in G.nodes(data=True):
    if ndata['subtype'] == 'H7N9' and len(G.in_edges(node)) != 0 and ndata['collection_date'].year == 2013:
        for source, sink, edata in G.in_edges(node, data=True):
            print(source, sink, edata['pwi'])

('A/environment/Nanjing/2913/2013', 'A/environment/Shanghai/S1438/2013', 7.9187609999999999)
('A/Wuxi/4/2013', 'A/chicken/Jiangsu/SC537/2013', 7.9577600000000004)
('A/duck/Jiangxi/3286/2009', 'A/Anhui/DEWH72-03/2013', 7.2299199999999999)
('A/duck/Jiangxi/3283/2009', 'A/Anhui/DEWH72-03/2013', 7.2299199999999999)
('A/Anhui/1-DEWH730/2013', 'A/chicken/Shanghai/S1080/2013', 7.8904420000000002)
('A/Anhui/DEWH72-01/2013', 'A/chicken/Shanghai/S1080/2013', 7.8904420000000002)
('A/duck/Jiangxi/3286/2009', 'A/Anhui/DEWH72-09/2013', 7.2299199999999999)
('A/duck/Jiangxi/3283/2009', 'A/Anhui/DEWH72-09/2013', 7.2299199999999999)
('A/Shanghai/MH01/2013', 'A/environment/Fujian/SC337/2013', 7.9906329999999999)
('A/chicken/Shanghai/S1078/2013', 'A/environment/Henan/SC232/2013', 7.9922029999999999)
('A/pigeon/Shanghai/S1423/2013', 'A/homing pigeon/Jiangsu/SD184/2013', 7.9955130000000008)
('A/chicken/Zhejiang/SD033/2013', 'A/duck/Zhejiang/SC410/2013', 7.9726350000000004)
('A/pigeon/Shanghai/S1069/2013', '

In [24]:
len(subgraphs[16].nodes())

77

In [25]:
# Let's find out if there are swine-->human transmissions in this graph
goi = nx.DiGraph(subgraphs[16])
for source, sink in goi.edges():
    if goi.node[source]['collection_date'] >= G.node[sink]['collection_date']:
        goi.remove_edge(source, sink)

for source, sink, data in goi.edges(data=True):
    